<a href="https://colab.research.google.com/github/AARABAabdallah/RNAclassification/blob/master/RNA_without_padding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import tnrange
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from librosa.util import pad_center
from time import time
import datetime
import pickle

In [0]:
data = pd.read_csv('drive/My Drive/Colab Workspace/RNA_classification/DataSet/All_RNA_Families_IdSeqTotal_IdFamily_Seq.csv')

In [0]:
#Split data
X = np.array(data.sequence)
y = np.array(data.id_family)
family_labels = np.unique(np.array(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [5]:
print(len(X_train))
print(len(X_test))

54039
18013


In [6]:
#Données de validations !
X_val1, X_val, y_val1, y_val = train_test_split(X_test, y_test, test_size=0.17)
len(X_val)

3063

In [0]:
X_train_copy = np.copy(X_train)
X_test_copy = np.copy(X_test)
X_val = np.copy(X_val)

In [0]:
X_train = np.copy(X_train_copy)
X_test = np.copy(X_test_copy)
X_val = np.copy(X_val)

In [0]:
 #Turning data into one hot coding
 
 def oneHot_encode(data):
  if isinstance(data,str) : # input is a string
    input_list = np.array(list(data))
    output = []
    for i in range(len(input_list)):
      if input_list[i] == 'A':
        output.append(np.array([1, 0, 0, 0]))
      elif input_list[i] == 'C':
        output.append(np.array([0, 1, 0, 0]))
      elif input_list[i] == 'G':
        output.append(np.array([0, 0, 1, 0]))
      elif input_list[i] == 'T':
        output.append(np.array([0, 0, 0, 1]))
      #else :    #Padding !!
      #  output.append(np.array([0, 0, 0, 0]))
    return output
  else: # input is an array of strings
    return np.array([ np.array(oneHot_encode(data[i])) for i in tnrange(int(data.shape[0]))])
 

In [10]:
X_train = oneHot_encode(X_train)
X_test = oneHot_encode(X_test)
X_val = oneHot_encode(X_val)

In [0]:
y_train_copy = np.copy(y_train)
y_test_copy = np.copy(y_test)
y_val_copy = np.copy(y_val)

In [0]:
y_train = np.copy(y_train_copy)
y_test = np.copy(y_test_copy)
y_val = np.copy(y_val_copy)

In [13]:
print(len(np.unique(y_train)))
print(len(np.unique(y_val)))

3016
1189


In [0]:
# OneHot Encoding labels: Just for now !!!
def one_hot_encode_labels(labels):
  output = []
  for i in tnrange(len(labels)):
    output_i = np.zeros(len(family_labels), dtype=int)
    output_i[labels[i]] = 1
    output.append(output_i)
  return np.array(output)

In [15]:
y_train = one_hot_encode_labels(y_train)
y_test = one_hot_encode_labels(y_test)
y_val = one_hot_encode_labels(y_val)

In [21]:
# Model Creation
from keras.models import Sequential
from keras.layers import LSTM, Input, Dense, Activation, Embedding

model = Sequential()

#model.add(Embedding(input_dim=4, output_dim=16, input_length=None))
model.add(LSTM(32, return_sequences=False, input_shape=(None, 4)))
model.add(Dense(len(family_labels),name='out_layer', activation='softmax'))
model.summary(90)

Model: "sequential_2"
__________________________________________________________________________________________
Layer (type)                            Output Shape                        Param #       
lstm_2 (LSTM)                           (None, 32)                          4736          
__________________________________________________________________________________________
out_layer (Dense)                       (None, 3016)                        99528         
Total params: 104,264
Trainable params: 104,264
Non-trainable params: 0
__________________________________________________________________________________________


In [0]:
X_train_init = X_train
y_train_init = y_train
def train_generator():
    i = 0
    #while i < len(X_train_init):
    while True:
        if i == len(X_train_init):
          i = 0
        x_tr = (X_train_init[i]).reshape(1, len(X_train_init[i]), 4)
        y_tr = y_train_init[i].reshape(1,len(y_train_init[i]))
        yield x_tr, y_tr
        i += 1

In [22]:
# model training with train_generator ==> without batch optimization
model.compile(loss='categorical_crossentropy', optimizer='adam')
history = model.fit(train_generator(), steps_per_epoch=len(X_train_init), epochs=1, verbose=1)#, validation_data=(X_val, y_val))

Epoch 1/1
 8668/54039 [===>..........................] - ETA: 47:52 - loss: 6.9801

KeyboardInterrupt: ignored

In [0]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
model.save_weights("drive/My Drive/Colab Workspace/RNA_classification/models/LSTM_OneHot_without_padding.h5")
history_file_name = 'drive/My Drive/Colab Workspace/RNA_classification/models/trainHistoryDict_without_padding'
with open(history_file_name, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

In [0]:
#from tqdm import tnrange
def model_accuracy(data_test, labels_test):
  acc = 0
  for i in tnrange(len(data_test), desc='testing progress'):
    if np.argmax(model.predict(data_test[i].reshape(1, len(X_test[i]), 4))) == np.argmax(labels_test[i]):
      acc += 1
  return (acc/len(data_test))*100

In [0]:
accuracy = model_accuracy(X_test[:5000], y_test[:5000])
print("the model's accuracy is: ", accuracy, "%")